## Chatbot interactivo Phi 3 Mini 4K Instruct con Whisper

### Introducción:
El Chatbot interactivo Phi 3 Mini 4K Instruct es una herramienta que permite a los usuarios interactuar con la demostración de Microsoft Phi 3 Mini 4K Instruct utilizando texto o entrada de audio. El chatbot puede ser utilizado para una variedad de tareas, como traducción, actualizaciones del clima y recopilación de información general.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Crear tu Token de Acceso de Huggingface](https://huggingface.co/settings/tokens)

Crea un nuevo token  
Proporciona un nuevo nombre  
Selecciona permisos de escritura  
Copia el token y guárdalo en un lugar seguro


El siguiente código en Python realiza dos tareas principales: importar el módulo `os` y establecer una variable de entorno.

1. Importar el módulo `os`:
   - El módulo `os` en Python proporciona una forma de interactuar con el sistema operativo. Permite realizar diversas tareas relacionadas con el sistema operativo, como acceder a variables de entorno, trabajar con archivos y directorios, etc.
   - En este código, el módulo `os` se importa utilizando la instrucción `import`. Esta instrucción hace que la funcionalidad del módulo `os` esté disponible para su uso en el script de Python actual.

2. Establecer una variable de entorno:
   - Una variable de entorno es un valor que puede ser accedido por programas que se ejecutan en el sistema operativo. Es una forma de almacenar configuraciones o información que puede ser utilizada por múltiples programas.
   - En este código, se está estableciendo una nueva variable de entorno utilizando el diccionario `os.environ`. La clave del diccionario es `'HF_TOKEN'`, y el valor se asigna desde la variable `HUGGINGFACE_TOKEN`.
   - La variable `HUGGINGFACE_TOKEN` se define justo encima de este fragmento de código y se le asigna un valor de cadena `"hf_**************"` utilizando la sintaxis `#@param`. Esta sintaxis se utiliza frecuentemente en Jupyter notebooks para permitir la entrada del usuario y la configuración de parámetros directamente en la interfaz del notebook.
   - Al establecer la variable de entorno `'HF_TOKEN'`, esta puede ser accedida por otras partes del programa o por otros programas que se ejecuten en el mismo sistema operativo.

En resumen, este código importa el módulo `os` y establece una variable de entorno llamada `'HF_TOKEN'` con el valor proporcionado en la variable `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Este fragmento de código define una función llamada clear_output que se utiliza para borrar la salida de la celda actual en Jupyter Notebook o IPython. Vamos a desglosar el código y entender su funcionalidad:

La función clear_output toma un parámetro llamado wait, que es un valor booleano. Por defecto, wait está configurado como False. Este parámetro determina si la función debe esperar hasta que haya nueva salida disponible para reemplazar la salida existente antes de borrarla.

La función en sí se utiliza para borrar la salida de la celda actual. En Jupyter Notebook o IPython, cuando una celda produce una salida, como texto impreso o gráficos, esa salida se muestra debajo de la celda. La función clear_output permite borrar esa salida.

La implementación de la función no se proporciona en el fragmento de código, como se indica con los puntos suspensivos (...). Los puntos suspensivos representan un marcador de posición para el código real que realiza el borrado de la salida. La implementación de la función puede implicar interactuar con la API de Jupyter Notebook o IPython para eliminar la salida existente de la celda.

En general, esta función ofrece una manera conveniente de borrar la salida de la celda actual en Jupyter Notebook o IPython, facilitando la gestión y actualización de la salida mostrada durante sesiones de codificación interactivas.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Realizar texto a voz (TTS) utilizando el servicio Edge TTS. Vamos a revisar las implementaciones de las funciones relevantes una por una:

1. `calculate_rate_string(input_value)`: Esta función toma un valor de entrada y calcula la cadena de velocidad para la voz TTS. El valor de entrada representa la velocidad deseada del habla, donde un valor de 1 representa la velocidad normal. La función calcula la cadena de velocidad restando 1 del valor de entrada, multiplicándolo por 100 y luego determinando el signo según si el valor de entrada es mayor o igual a 1. La función devuelve la cadena de velocidad en el formato "{signo}{velocidad}".

2. `make_chunks(input_text, language)`: Esta función toma un texto de entrada y un idioma como parámetros. Divide el texto de entrada en fragmentos según las reglas específicas del idioma. En esta implementación, si el idioma es "English", la función divide el texto en cada punto (".") y elimina cualquier espacio en blanco al principio o al final. Luego, agrega un punto a cada fragmento y devuelve la lista filtrada de fragmentos.

3. `tts_file_name(text)`: Esta función genera un nombre de archivo para el archivo de audio TTS basado en el texto de entrada. Realiza varias transformaciones en el texto: elimina un punto final (si está presente), convierte el texto a minúsculas, elimina espacios en blanco al principio y al final, y reemplaza los espacios con guiones bajos. Luego, trunca el texto a un máximo de 25 caracteres (si es más largo) o utiliza el texto completo si está vacío. Finalmente, genera una cadena aleatoria utilizando el módulo [`uuid`] y la combina con el texto truncado para crear el nombre del archivo en el formato "/content/edge_tts_voice/{texto_truncado}_{cadena_aleatoria}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Esta función combina varios archivos de audio en un solo archivo de audio. Toma una lista de rutas de archivos de audio y una ruta de salida como parámetros. La función inicializa un objeto vacío `AudioSegment` llamado [`merged_audio`]. Luego, itera a través de cada ruta de archivo de audio, carga el archivo de audio utilizando el método `AudioSegment.from_file()` de la biblioteca `pydub`, y agrega el archivo de audio actual al objeto [`merged_audio`]. Finalmente, exporta el audio combinado a la ruta de salida especificada en formato MP3.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Esta función realiza la operación TTS utilizando el servicio Edge TTS. Toma una lista de fragmentos de texto, la velocidad del habla, el nombre de la voz y la ruta de guardado como parámetros. Si el número de fragmentos es mayor que 1, la función crea un directorio para almacenar los archivos de audio individuales de los fragmentos. Luego, itera a través de cada fragmento, construye un comando Edge TTS utilizando la función `calculate_rate_string()`, el nombre de la voz y el texto del fragmento, y ejecuta el comando utilizando la función `os.system()`. Si la ejecución del comando es exitosa, agrega la ruta del archivo de audio generado a una lista. Después de procesar todos los fragmentos, combina los archivos de audio individuales utilizando la función `merge_audio_files()` y guarda el audio combinado en la ruta de guardado especificada. Si solo hay un fragmento, genera directamente el comando Edge TTS y guarda el audio en la ruta de guardado. Finalmente, devuelve la ruta de guardado del archivo de audio generado.

6. `random_audio_name_generate()`: Esta función genera un nombre de archivo de audio aleatorio utilizando el módulo [`uuid`]. Genera un UUID aleatorio, lo convierte en una cadena, toma los primeros 8 caracteres, agrega la extensión ".mp3" y devuelve el nombre de archivo de audio aleatorio.

7. `talk(input_text)`: Esta función es el punto de entrada principal para realizar la operación TTS. Toma un texto de entrada como parámetro. Primero verifica la longitud del texto de entrada para determinar si es una oración larga (mayor o igual a 600 caracteres). Según la longitud y el valor de la variable `translate_text_flag`, determina el idioma y genera la lista de fragmentos de texto utilizando la función `make_chunks()`. Luego, genera una ruta de guardado para el archivo de audio utilizando la función `random_audio_name_generate()`. Finalmente, llama a la función `edge_free_tts()` para realizar la operación TTS y devuelve la ruta de guardado del archivo de audio generado.

En general, estas funciones trabajan juntas para dividir el texto de entrada en fragmentos, generar un nombre de archivo para el archivo de audio, realizar la operación TTS utilizando el servicio Edge TTS y combinar los archivos de audio individuales en un solo archivo de audio.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

La implementación de dos funciones: convert_to_text y run_text_prompt, así como la declaración de dos clases: str y Audio.

La función convert_to_text toma un audio_path como entrada y transcribe el audio a texto utilizando un modelo llamado whisper_model. Primero, la función verifica si la bandera gpu está configurada como True. Si es así, se utiliza el whisper_model con ciertos parámetros como word_timestamps=True, fp16=True, language='English' y task='translate'. Si la bandera gpu es False, el whisper_model se utiliza con fp16=False. La transcripción resultante se guarda en un archivo llamado 'scan.txt' y se devuelve como texto.

La función run_text_prompt toma un mensaje y un chat_history como entrada. Utiliza la función phi_demo para generar una respuesta de un chatbot basada en el mensaje de entrada. La respuesta generada se pasa a la función talk, que convierte la respuesta en un archivo de audio y devuelve la ruta del archivo. La clase Audio se utiliza para mostrar y reproducir el archivo de audio. El audio se muestra utilizando la función display del módulo IPython.display, y el objeto Audio se crea con el parámetro autoplay=True, para que el audio comience a reproducirse automáticamente. El chat_history se actualiza con el mensaje de entrada y la respuesta generada, y se devuelven una cadena vacía y el chat_history actualizado.

La clase str es una clase incorporada en Python que representa una secuencia de caracteres. Proporciona varios métodos para manipular y trabajar con cadenas, como capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, y más. Estos métodos te permiten realizar operaciones como buscar, reemplazar, formatear y manipular cadenas.

La clase Audio es una clase personalizada que representa un objeto de audio. Se utiliza para crear un reproductor de audio en el entorno de Jupyter Notebook. La clase acepta varios parámetros como data, filename, url, embed, rate, autoplay y normalize. El parámetro data puede ser un array de numpy, una lista de muestras, una cadena que representa un nombre de archivo o URL, o datos PCM sin procesar. El parámetro filename se utiliza para especificar un archivo local desde el cual cargar los datos de audio, y el parámetro url se utiliza para especificar una URL desde la cual descargar los datos de audio. El parámetro embed determina si los datos de audio deben incrustarse utilizando un URI de datos o referenciarse desde la fuente original. El parámetro rate especifica la tasa de muestreo de los datos de audio. El parámetro autoplay determina si el audio debe comenzar a reproducirse automáticamente. El parámetro normalize especifica si los datos de audio deben normalizarse (reescalarse) al rango máximo posible. La clase Audio también proporciona métodos como reload para recargar los datos de audio desde un archivo o URL, y atributos como src_attr, autoplay_attr y element_id_attr para recuperar los atributos correspondientes del elemento de audio en HTML.

En general, estas funciones y clases se utilizan para transcribir audio a texto, generar respuestas de audio desde un chatbot y mostrar y reproducir audio en el entorno de Jupyter Notebook.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
